<a href="https://colab.research.google.com/github/steveding1/DE-learning/blob/main/json_to_csv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [67]:
import pandas as pd

In [68]:
def assignment_to_df(assignment):
  df = pd.DataFrame(assignment['properties'])
  df['type'] = assignment['type']
  print('assignment Dict converted to a DataFrame')
  return df

def rectangle(item):
  return (item["x2"]-item["x1"])*(item["y2"]-item["y1"])

def convert_to_csv(assignment, annotations) -> pd.DataFrame:
  rows = []
  object_ids = []
  assignmentdf = assignment_to_df(assignment)

  #turn annotations Json to a Row format:
  for annotation in annotations:
    keys = ['object_id', 'rectangle', 'properties']
    for key,item in annotation.items():
      #check if the key is in ['object_id', 'rectangle', 'properties'] and not repeated
      if key == keys[0]:
        row = {key:item}
        print(f'processing object_id {item}...')
        if item in object_ids:
          print(f'WARNING: found duplicate object_id: {item}!')
        else:
          object_ids.append(item)
      elif key == keys[1]:
        row["area"] = rectangle(item)
      elif key == keys[2]:
        if len(item)!=3:
          raise ValueError('There should be exact 3 properties for each object_id!')
        for properti in item:   #properti = {'property_id': 1,'value': '自行车1',}
          tempdf = assignmentdf[assignmentdf['id']==properti['property_id']]
          #use id value to find the name in the 'properties' as the column name
          value = properti['value']
          #check if the value is in assignment['properties']['items']
          #print(tempdf['items'].to_list()[0])
          if value in tempdf['items'].to_list()[0]:
            row[tempdf['name'].to_string(index=False)] = value
          else:
            raise ValueError(
                f'The value "{value}" is not in items under properties in the assignment Dict: {tempdf["items"].to_string(index=False)}'
            )
      else:
        raise ValueError('annotations dict has the key:',key,' that is not in ',str(keys))
    rows.append(row)

  df = pd.DataFrame(rows)
  df = pd.merge(df,df.groupby('object_type')['object_type'].count().rename(index='object_total'),left_on=['object_type'],right_index=True)
  print('Dict Annotations has been converted to csv format successfully')
  return df

In [69]:
if __name__ == '__main__':
    # Do not change the input data
    assignment = {
        'type': 'rectangle',
        'properties': [{
            'id': 1,
            'name': 'object_type',
            'items': ['car', 'big car', 'person', 'bicycle'],
        }, {
            'id': 2,
            'name': 'Is_clear',
            'items': ['Yes', 'No'],
        }, {
            'id': 3,
            'name': 'Is_complete',
            'items': ['complete', 'incomplete'],
        }],
    }

    annotations = [{
        'object_id': 100,
        'rectangle': {
            'x1': 100,
            'y1': 100,
            'x2': 150,
            'y2': 150,
        },
        'properties': [{
            'property_id': 1,
            'value': 'bicycle',
        }, {
            'property_id': 2,
            'value': 'Yes',
        }, {
            'property_id': 3,
            'value': 'complete',
        }]
    }, {
        'object_id': 101,
        'rectangle': {
            'x1': 200,
            'y1': 100,
            'x2': 450,
            'y2': 150,
        },
        'properties': [{
            'property_id': 1,
            'value': 'big car',
        }, {
            'property_id': 2,
            'value': 'Yes',
        }, {
            'property_id': 3,
            'value': 'incomplete',
        }]
    }, {
        'object_id': 102,
        'rectangle': {
            'x1': 100,
            'y1': 150,
            'x2': 120,
            'y2': 150,
        },
        'properties': [{
            'property_id': 3,
            'value': 'complete',
        }, {
            'property_id': 1,
            'value': 'big car',
        }, {
            'property_id': 2,
            'value': 'No',
        }]
    }, {
        'object_id': 103,
        'rectangle': {
            'x1': 300,
            'y1': 100,
            'x2': 400,
            'y2': 200,
        },
        'properties': [{
            'property_id': 3,
            'value': 'complete',
        }, {
            'property_id': 1,
            'value': 'person',
        }, {
            'property_id': 2,
            'value': 'Yes',
        }]
    }
, {
        'object_id': 103,
        'rectangle': {
            'x1': 300,
            'y1': 100,
            'x2': 400,
            'y2': 200,
        },
        'properties': [{
            'property_id': 3,
            'value': 'complete',
        }, {
            'property_id': 1,
            'value': 'person',
        }, {
            'property_id': 2,
            'value': 'Yes',
        }]
    }]

    df = convert_to_csv(assignment, annotations)
    display(df)
    # df.to_csv('annotations.csv')

assignment Dict converted to a DataFrame
processing object_id 100...
processing object_id 101...
processing object_id 102...
processing object_id 103...
processing object_id 103...
Dict Annotations has been converted to csv format successfully


,object_id,area,object_type,Is_clear,Is_complete,object_total
0,100,2500,bicycle,Yes,complete,1
1,101,12500,big car,Yes,incomplete,2
2,102,0,big car,No,complete,2
3,103,10000,person,Yes,complete,2
4,103,10000,person,Yes,complete,2
